# Group Details

## Group Name: Group 08

### Student 1: Jasper Linders

### Student 2: Alexander Liu

### Student 3: Sjoerd van Straten

# Loading Data and Preliminaries

In [40]:
# from google.colab import drive
# drive.mount('/content/drive')

In [41]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [43]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [44]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""
example = load_array('data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


In [45]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time.
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_array('data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-5.85725792 -5.394571           nan         nan]
 [-6.03781257 -5.72445953         nan         nan]
 [-0.90623054 -6.93416278         nan         nan]
 [ 2.83149339 -7.50100819         nan         nan]
 [-2.85586881  1.77667501         nan         nan]
 [ 4.04424526  4.00563603         nan         nan]
 [-5.24887713 -4.83081005         nan         nan]
 [-5.81391023 -5.1109838          nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


# Data Handling and Preprocessing

In [46]:
import os
from torch.utils.data import Dataset, DataLoader, TensorDataset

class ImportData(Dataset):
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.file_list = sorted(os.listdir(folder_path))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        file_name = self.file_list[index]
        file_path = os.path.join(self.folder_path, file_name)
        data, label = load_array(file_path, task='task 1')
        return data, label

# Create an instance of the custom dataset class with the folder path
train_import = ImportData('data/task 1/train/')
test_import = ImportData('data/task 1/test/')

X_train_import = []
y_train_import = []
X_test_import = []
y_test_import = []

# Iterate through the train_dataset to extract data and labels
for data, label in train_import:
    X_train_import.append(data)
    y_train_import.append(label)

for data, label in test_import:
    X_test_import.append(data)
    y_test_import.append(label)

max_length = 9

# Pad the data samples with zeros to have the same shape
X_train_padded = []
for data in X_train_import:
    pad_width = max_length - data.shape[0]
    padded_data = np.pad(data, ((0, pad_width), (0, 0)), mode='constant')
    X_train_padded.append(padded_data)

y_train_padded = []
for label in y_train_import:
    pad_width = max_length - label.shape[0]
    padded_label = np.pad(label, ((0, pad_width), (0, 0)), mode='constant')
    y_train_padded.append(padded_label)

X_test_padded = []
for data in X_test_import:
    pad_width = max_length - data.shape[0]
    padded_data = np.pad(data, ((0, pad_width), (0, 0)), mode='constant')
    X_test_padded.append(padded_data)

y_test_padded = []
for label in y_test_import:
    pad_width = max_length - label.shape[0]
    padded_label = np.pad(label, ((0, pad_width), (0, 0)), mode='constant')
    y_test_padded.append(padded_label)

# Convert the padded data and labels to tensors
X_train = torch.tensor(np.array(X_train_padded))
y_train = torch.tensor(np.array(y_train_padded))
X_test = torch.tensor(np.array(X_test_padded))
y_test = torch.tensor(np.array(y_test_padded))

# Print the shape of X_train and the first label in y_train
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

torch.Size([900, 9, 5])
torch.Size([900, 9, 2])
torch.Size([100, 9, 5])
torch.Size([100, 9, 2])


In [57]:
print(X_train[0])
print(y_train[0])

tensor([[ 4.5467, -5.2712,  5.0786, -1.0969, -3.6693],
        [ 3.5751,  4.3271, -0.0810, -0.5787, -2.9597],
        [ 2.6773, -5.1597,  5.3524,  2.4265,  1.4587],
        [ 2.2746, -6.7958,  2.2963,  0.3042, -0.4066],
        [ 2.3836, -2.7586,  1.7794, -0.0764, -0.3595],
        [ 4.2860,  0.3498,  4.8286,  1.3563, -2.5576],
        [ 3.0312, -0.5056, 15.0463, -0.9477, 14.9813],
        [ 4.1353,  1.8134,  3.8438, -0.6990, -2.2509],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[ 0.1560, -5.7285],
        [-1.0527, -9.6791],
        [-0.4624, -7.3590],
        [-0.3819, -6.3529],
        [-6.3564,  2.7568],
        [ 0.7670, -5.2778],
        [-5.2653, 89.5328],
        [-0.9416, -7.6763],
        [ 0.0000,  0.0000]], dtype=torch.float64)


In [47]:
# train_dataset = TensorDataset(X_train, y_train)
# test_dataset = TensorDataset(X_test, y_test)

# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [70]:
import torch
from torch.utils.data import Dataset, DataLoader

# Function to process a single data point
def create_graph(data_point):

    #Create node features
    node_features = data_point[:, 1:]  # Exclude the mass column

    # Compute edge indices
    edge_indices = []
    for i in range(9):
        for j in range(9):
            if i != j:
                edge_indices.append([i, j])
    edge_indices = torch.tensor(edge_indices).t().contiguous()

    # # Compute edge features
    # edge_features = []
    # for i in range(9):
    #     for j in range(9):
    #         if i < j:
    #             relative_position = node_features[j] - node_features[i]
    #             relative_mass = node_features[j, 0] / node_features[i, 0]
    #             edge_features.append(torch.cat((torch.tensor([relative_position]), torch.tensor([relative_mass], dtype=torch.float32))))
    # edge_features = torch.stack(edge_features)

    return node_features, edge_indices #, edge_features

node_features, edge_indices = create_graph(initial_state)

In [71]:
edge_indices

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
         3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5,
         6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8],
        [1, 2, 3, 4, 5, 6, 7, 8, 0, 2, 3, 4, 5, 6, 7, 8, 0, 1, 3, 4, 5, 6, 7, 8,
         0, 1, 2, 4, 5, 6, 7, 8, 0, 1, 2, 3, 5, 6, 7, 8, 0, 1, 2, 3, 4, 6, 7, 8,
         0, 1, 2, 3, 4, 5, 7, 8, 0, 1, 2, 3, 4, 5, 6, 8, 0, 1, 2, 3, 4, 5, 6, 7]])

In [74]:
from torch_geometric.data import Data

train_data = Data(x=X_train, edge_index=edge_indices, y=y_train)
test_data = Data(x=X_test, edge_index=edge_indices, y=y_test)

# # Add additional arguments to `data`:
# data.train_idx = torch.tensor([...], dtype=torch.long)
# data.test_mask = torch.tensor([...], dtype=torch.bool)

# Analyzing the graph structure:
print(train_data.num_nodes)
print(test_data.num_nodes)

# >>> 23

print(train_data.is_directed())
print(test_data.is_directed())

# >>> False

# PyTorch tensor functionality:
train_data = train_data.pin_memory()
train_data = train_data.to('cuda:0', non_blocking=True)

900
100
False
False


In [49]:
# import networkx as nx
# import matplotlib.pyplot as plt

# value, label = train_dataloader.dataset[1]
# node_features, edge_indices, edge_features = create_graph(value)

# # Create an empty graph
# graph = nx.Graph()

# # Add nodes with node numbers as labels
# for i in range(node_features.shape[0]):
#     graph.add_node(i, label=i)

# # Add edges
# for edge in edge_indices.t().tolist():
#     graph.add_edge(*edge)

# # Plot the graph
# pos = nx.spring_layout(graph, seed=42)  # Layout for node positioning
# nx.draw_networkx(graph, pos=pos, with_labels=True)

# # Add node numbers as text labels
# labels = nx.get_node_attributes(graph, "label")
# nx.draw_networkx_labels(graph, pos=pos, labels=labels)

# # Add node features as text labels
# for i, feature in enumerate(node_features):
#     x, y = feature[:2]
#     plt.text(pos[i][0], pos[i][1], f"({x:.2f}, {y:.2f})", ha="center", va="bottom")

# # Display the plot
# plt.axis("off")
# plt.show()

# Model Implementation

In [50]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class GraphConvolution(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(GraphConvolution, self).__init__()
#         self.weight = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
#         self.bias = nn.Parameter(torch.FloatTensor(output_dim))
#         self.reset_parameters()

#     def reset_parameters(self):
#         nn.init.xavier_uniform_(self.weight)
#         nn.init.zeros_(self.bias)

#     def forward(self, x, adj):
#         support = torch.matmul(x, self.weight)
#         output = torch.matmul(adj, support) + self.bias
#         return output


# class GCN(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(GCN, self).__init__()
#         self.gc1 = GraphConvolution(input_dim, hidden_dim)
#         self.gc2 = GraphConvolution(hidden_dim, output_dim)
#         self.relu = nn.ReLU()

#     def forward(self, x, adj):
#         x = self.relu(self.gc1(x, adj))
#         x = self.gc2(x, adj)
#         return x

In [51]:
from torch_geometric.nn.conv import GatedGraphConv as GGC

class GGCN(nn.Module):
    def __init__(self, out_channels, num_layers, aggr, bias):
        super(GGCN, self).__init__()
        self.gc1 = GGC(out_channels, num_layers, aggr, bias)
        self.relu = nn.ReLU()

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        return x

# Model Training

In [73]:
'''
Training the GNN
'''
model_gnn = GGCN(out_channels=2, num_layers=5, aggr='add', bias=True) # initialize GNN
print(model_gnn)

# same loss and optimizer as before
loss_func = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model_gnn.parameters(), lr=0.01, weight_decay=5e-4)

def train_gnn():
    model_gnn.train()  # set the model to training 'mode' (i.e., apply dropout)
    optimizer.zero_grad()  # set gradients to 0
    out = model_gnn(train_data)  # propagate the data through the model
    loss = loss_func(out[data.train_mask], data.y[data.train_mask])  # compute the loss based on our training mask
    loss.backward()  # derive gradients
    optimizer.step()  # update all parameters based on the gradients
    return loss

def test_gnn(mask):
    model_gnn.eval()  # set the model to evaluation 'mode' (don't use dropout)
    out = model_gnn(data.x, data.edge_index)  # propagate the data through the model
    pred = out.argmax(dim=1)  # as prediction, we take the class with the highest probability
    test_correct = pred[mask] == data.y[mask]  # create a tensor that evaluates whether predictions were correct
    test_acc = int(test_correct.sum()) / int(mask.sum())  # get the accuracy
    return test_acc


train_accs = []
test_accs = []
epochs = 50
for epoch in range(1, epochs+1): 
    loss = train_gnn()  # do one training step over the entire dataset
    train_acc = test_gnn(data.train_mask)  # compute the training accuracy
    test_acc = test_gnn(data.test_mask)  # compute the test accuracy
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    train_accs.append(train_acc)  # save accuracies so we can plot them
    test_accs.append(test_acc)

GGCN(
  (gc1): GatedGraphConv(2, num_layers=5)
  (relu): ReLU()
)


TypeError: GGCN.forward() missing 1 required positional argument: 'adj'

# Evaluation

In [ ]:
#todo